## Altmap Experiments
### Compare altmap to map eq using networkx


In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict

plt.rcParams.update({'font.size': 14})
%pylab

%run helpers.py
# loads the following helper functions:
# infomap(net_path, altmap=False, additional_args='')
# read_tree(tree_path)
# drawNetwork(G, communities)
# altmap_cost(G, communities)
# create_initfile(G, N_partitions=None, randomized=True)

def generate_two_rings(n_ring=10):
    N = 2 * n_ring  # num nodes
    G = nx.MultiGraph()
    G.add_nodes_from(range(1, N + 1))

    for n in range(1, N):
        G.add_edge(n, n + 1, weight=1)

    G.add_edge(1, int(N / 2), weight=1)
    G.add_edge(int(N / 2) + 1, N, weight=1)
    return G


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [4]:
n_ring = 100
N = 2*n_ring # num nodes
G = generate_two_rings(n_ring)

# init
communities = {}
for n in range(1, N+1 ):
    communities[n] = n
    
cost = altmap_cost(G, communities)
print (f'Initial Cost L = {cost}\n')

# ground truth
communities = {}
for n in range(1, int(N / 2)+1 ):
    communities[n] = 1
    
for n in range(int(N / 2)+1, N+1 ):
    communities[n] = 2

cost = altmap_cost(G, communities)
print (f'Ground Truth Cost L = {cost}\n')

# 2 mixed cliques
communities = {}
for n in range(1, int(N / 4)+1 ):
    communities[n] = 1

for n in range(int(N / 2) + 1, int(N / 2) + int(N / 4) + 1 ):
    communities[n] = 1
    
for n in range(int(N / 4)+1, int(N / 2) + 1 ):
    communities[n] = 2
    
for n in range(int(N / 2) + int(N / 4) + 1, N + 1 ):
    communities[n] = 2

cost = altmap_cost(G, communities)
print (f'Mixed Communities Cost L = {cost}\n')

# 4 mixed cliques
communities = {}
for n in range(1, int(N / 4)+1 ):
    communities[n] = 1

for n in range(int(N / 2) + 1, int(N / 2) + int(N / 4) + 1 ):
    communities[n] = 2
    
for n in range(int(N / 4)+1, int(N / 2) + 1 ):
    communities[n] = 3
    
for n in range(int(N / 2) + int(N / 4) + 1, N + 1 ):
    communities[n] = 4

cost = altmap_cost(G, communities)
print (f'Four Mixed Communities Cost L = {cost}\n')

Initial Cost L = -0.00724344653766134

Ground Truth Cost L = -0.9574362123777358

Mixed Communities Cost L = -0.8367058739707411

Four Mixed Communities Cost L = -1.7993140655681912



In [5]:
# compute essential cost function values for a network of two rings with
# given ring size
def two_rings_cost(ring_size = 3, print_output=False):
    n = ring_size
    m = 2*n+1 # number of edges in the network

    J_init = ((m-3)*np.log2(m-1) + 3*np.log2(2*m-3) - 3) / m - np.log2(m)
    J_true = np.log2(m) - 1.0 - (m-1) / m * np.log2(m-1)
    J_ind = -1.0
    J_lower_bound = -(3 - 3*np.log2(3) + m*np.log2(m))/m
    
    p1 =n/(2*m); p2 = (n+1)/(2*m)
    p1not1 = 1/m; p11 = p1 - p1not1; 
    p2not2 = 3/(2*m); p22 = p2 - p2not2; 
    J_4_comms = -2*(p11*np.log2(p11 / (p1**2)) + p1not1*np.log2(p1not1 / (p1*(1-p1))) +
                    p22*np.log2(p22 / (p2**2)) + p2not2*np.log2(p2not2 / (p2*(1-p2))))
    
    if print_output:
        print (f"\n2 Rings network with nc = {nc} nodes per clique:\n")
        print (f"Each node a module - cost = {J_init}")
        print (f"Ground truth cost = {J_true}")
        print (f"Independent sets cost = {J_ind}\n")
        print (f"4 communities = {J_4_comms}\n")
        print (f"Lower bound = {J_lower_bound}\n")
    
    return J_init, J_true, J_ind, J_4_comms

n_max = 50
n_list = list(range(3, n_max + 1))
J_init_list = np.zeros((len(n_list), 1))
J_true_list = np.zeros((len(n_list), 1))
J_ind_list = np.zeros((len(n_list), 1))
J_4comms_list = np.zeros((len(n_list), 1))
for i, n in enumerate(n_list):
    J_init_list[i], J_true_list[i], J_ind_list[i], J_4comms_list[i] = two_rings_cost(n)

plt.figure()
plt.plot(n_list, J_init_list, 'b', label='Initial cost (N ind sets)')
plt.plot(n_list, J_true_list, 'm', label='Ground truth cost')
plt.plot(n_list, J_ind_list, 'r', label='2 Independent sets cost')
plt.plot(n_list, J_4comms_list, 'c', label='4 Communities')
plt.grid()
plt.title('2 Rings network - cost over size')
plt.xlabel('Nodes per clique')
plt.ylabel('Altmap cost')
plt.legend()

In [6]:
nx.write_pajek(G, workspace_path +  filename + '.net')
# infomap(workspace_path +  filename + '.net', altmap=False)
# infomap(workspace_path +  filename + '.net', altmap=True)
communities = create_initfile(G, randomized=True)
infomap(workspace_path +  filename + '.net', altmap=True, additional_args=' --cluster-data ./workspace/init.tree')

# read results
communities_found, num_communities_found = read_communities_from_tree_file()
print (communities_found)
print (f'We found {num_communities_found} communities.')

cost = altmap_cost(G, communities_found)
print (f'Achieved cost L = {cost}')


OrderedDict([(1, 5), (2, 5), (3, 5), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 5), (10, 5), (11, 5), (12, 5), (13, 5), (14, 5), (15, 5), (16, 6), (17, 6), (18, 6), (19, 1), (20, 1), (21, 1), (22, 1), (23, 6), (24, 6), (25, 2), (26, 2), (27, 2), (28, 2), (29, 2), (30, 2), (31, 2), (32, 2), (33, 2), (34, 2), (35, 2), (36, 2), (37, 2), (38, 3), (39, 3), (40, 3), (41, 3), (42, 3), (43, 1), (44, 1), (45, 6), (46, 6), (47, 6), (48, 6), (49, 6), (50, 4), (51, 4), (52, 4), (53, 4), (54, 4), (55, 4), (56, 1), (57, 1), (58, 5), (59, 5), (60, 5), (61, 3), (62, 3), (63, 3), (64, 3), (65, 3), (66, 3), (67, 3), (68, 3), (69, 3), (70, 3), (71, 3), (72, 3), (73, 7), (74, 7), (75, 7), (76, 7), (77, 7), (78, 7), (79, 7), (80, 7), (81, 7), (82, 6), (83, 6), (84, 6), (85, 6), (86, 6), (87, 7), (88, 7), (89, 7), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 5), (101, 5), (102, 6), (103, 6), (104, 6), (105, 6), (106, 6), (107, 6), (108, 6), (109, 6), (110,

In [7]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)